In [2]:
import pandas as pd

In [30]:
municipality = pd.read_csv("clean_data/turkey_municipality.csv")
parlimentary = pd.read_csv("clean_data/turkish_parlimentary.csv")
parlimentary_d = pd.read_csv("clean_data/milletvekili_detailed.csv")
party_alignment = pd.read_excel("clean_data/party_alignment.xlsx")
events = pd.read_excel("data/turkish_catastrophic_event.xlsx")

In [31]:
parlimentary = parlimentary.sort_values(by = ["year", "turnout"], ascending = False)
parlimentary = parlimentary.drop_duplicates(subset = ["year"], keep = "first")

# Municipality

In [107]:
municipality.drop(columns = ['prev_year',
       'winner', "prev_winner", "swing"], inplace = True)
municipality.city = municipality.city.str.replace("mersin", "icel")
municipality = municipality[(municipality.city != "turkiye") & (municipality.city != "altkademebelediyesi")]
municipality["percent_turnout"] = municipality["turnout"] / municipality["registered_voters"]

In [110]:
def determine_winner(row):
    row = row.dropna(axis=0, how='any')
    row = row.drop(["city", "year", "registered_voters", 
                       "turnout", "valid_vote", 
                       "percent_turnout"])
    row = row.astype(int)
    winner = row.idxmax()
    return winner

In [111]:
years = list(municipality.year.unique())
years = sorted(years, reverse = True)
municipality["prev_year"] = municipality.year.apply(lambda row: None if row == 1989 else years[years.index(row) + 1])

In [112]:
municipality["winner"] = municipality.apply(lambda row: determine_winner(row), axis = 1)
municipality["winner"] = municipality["winner"].str.split("_", expand = True)[0]

In [115]:
municipality_results = municipality[["city", "year", "winner"]].drop_duplicates()
municipality_results.rename(columns = {"winner": "prev_winner", "year": "prev_year"}, inplace = True)
municipality = municipality.merge(municipality_results, on = ["city", "prev_year"], how = "left")

In [116]:
municipality["swing"] = ((municipality["winner"] != municipality["prev_winner"]) & (municipality["prev_winner"].notna())).astype(int)

In [117]:
municipality.winner = municipality.winner.str.lower()
municipality.prev_winner = municipality.prev_winner.str.lower()
municipality.winner = municipality.winner.str.replace("ak parti", "akp")
municipality.prev_winner = municipality.prev_winner.str.replace("ak parti", "akp")

In [118]:
party_alignment.rename(columns = {"party": "winner"}, inplace = True)
party_alignment.winner = party_alignment.winner.str.replace("ak parti", "akp")
municipality = municipality.merge(party_alignment, on = "winner", how = "left")

In [119]:
party_alignment.columns = ["prev_winner", "nat_prev", "right_wing_prev", "left_wing_prev",
                           "political_islam_prev", "kurdish_prev", "liberal_prev", "socialist_prev",
                          "social_democ_prev", "populist_prev"]

In [121]:
municipality = municipality.merge(party_alignment, on = "prev_winner", how = "left")
municipality.to_csv("clean_data/turkey_municipality.csv", index = False)

# Parlimentary

In [32]:
parlimentary.drop(columns = ["num_milletvekili", "toplam_gecerli_oy_sayisi", "milletvekili_sayisi", "bgmztotal_votes"], inplace = True)
parlimentary.rename(columns = {"registered_voter": "registered_voters",
                             'valid_votes': "valid_votes"}, inplace = True)
cols = [i for i in parlimentary.columns if "percent_votes" not in i]
parlimentary = parlimentary[cols]
parlimentary.columns = [i.replace("total_votes", "") for i in parlimentary.columns]
parlimentary["city"] = ["turkiye"]*len(parlimentary)

In [33]:
years = list(parlimentary.year.unique())
years = sorted(years, reverse = True)
parlimentary["prev_year"] = parlimentary.year.apply(lambda row: None if row == "1950" else years[years.index(row) + 1])

In [34]:
def determine_winner_p(row):
    row = row.drop(["city", "year", "registered_voters", 
                       "turnout", "valid_vote", 
                       "percent_turnout"])
    row = row.dropna(axis=0, how='any')
    row = row.astype(int)
    winner = row.idxmax()
    return winner

In [35]:
parlimentary["winner"] = parlimentary.apply(lambda row: determine_winner_p(row), axis = 1)
parlimentary["winner"] = parlimentary["winner"].str.split("_", expand = True)[0]
parlimentary.winner = parlimentary.winner.str.replace("ak", "akp")

In [36]:
parlimentary_results = parlimentary[["year", "winner"]].drop_duplicates()
parlimentary_results.rename(columns = {"winner": "prev_winner", "year": "prev_year"}, inplace = True)
parlimentary = parlimentary.merge(parlimentary_results, on = ["prev_year"], how = "left")
parlimentary.loc[18, "prev_winner"] = "chp"

In [37]:
party_alignment.rename(columns = {"party": "winner"}, inplace = True)
parlimentary = parlimentary.merge(party_alignment, on = "winner", how = "left")

In [38]:
party_alignment.columns = ["prev_winner", "nat_prev", "right_wing_prev", "left_wing_prev",
                           "political_islam_prev", "kurdish_prev", "liberal_prev", "socialist_prev",
                          "social_democ_prev", "populist_prev"]
parlimentary = parlimentary.merge(party_alignment, on = "prev_winner", how = "left")

In [39]:
parlimentary["swing"] = ((parlimentary["winner"] != parlimentary["prev_winner"]) & (parlimentary["prev_winner"].notna())).astype(int)

In [40]:
parlimentary.winner = parlimentary.winner.str.lower()
parlimentary.prev_winner = parlimentary.prev_winner.str.lower()
parlimentary.winner = parlimentary.winner.str.replace("ak parti", "akp")
parlimentary.prev_winner = parlimentary.prev_winner.str.replace("ak parti", "akp")

# Concat Municipality and Parlimentary

In [41]:
parlimentary["elec_type"] = ["parlimentary"] * len(parlimentary)
municipality["elec_type"] = ["municipality"] * len(municipality)

In [42]:
elections_all = pd.concat([parlimentary, municipality], ignore_index = True)

In [43]:
elections_all.to_csv("clean_data/elections_all.csv", index = False)

In [44]:
elections_all = pd.read_csv("clean_data/elections_all.csv")

# Events

In [ ]:
events = pd.get_dummies(events)
events.to_csv("clean_data/turkish_crises.csv", index = False)

In [53]:
events = pd.read_csv("clean_data/turkish_crises.csv")

In [56]:
elections_all = elections_all.merge(events, on = "year", how = "left")

In [58]:
elections_all.to_csv("clean_data/elections_all.csv", index = False)

In [59]:
events

,year,crisis type_civil unrest,crisis type_coup,crisis type_economic,crisis type_refugee,crisis type_regufee,crisis type_soft coup,crisis type_terrorism,crisis type_war
0,1960,0,1,0,0,0,0,0,0
1,1970,1,0,0,0,0,0,0,0
2,1970,0,0,1,0,0,0,0,0
3,1971,0,0,0,0,0,1,0,0
4,1974,0,0,0,0,0,0,0,1
5,1979,0,0,1,0,0,0,0,0
6,1980,0,1,0,0,0,0,0,0
7,1984,0,0,0,0,0,0,0,1
8,1994,0,0,1,0,0,0,0,0
9,1997,0,0,0,0,0,1,0,0
